<a href="https://colab.research.google.com/github/tanakak11/DNN_WienerFilter/blob/main/DNN_WienerFilter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torch>=1.2.0
!pip install torchaudio

     |████████████████████████████████| 7.6MB 4.7MB/s 
     |████████████████████████████████| 776.8MB 19kB/s 
ERROR: torchvision 0.8.1+cu101 has requirement torch==1.7.0, but you'll have torch 1.7.1 which is incompatible.
  Found existing installation: torch 1.7.0+cu101
    Uninstalling torch-1.7.0+cu101:
      Successfully uninstalled torch-1.7.0+cu101


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch import Tensor
from torch.utils.data import Dataset

import torchaudio
from torchaudio import datasets, transforms
from torchaudio.datasets.utils import(
    download_url,
    extract_archive,
    walk_files,
)

import os
import glob
import pickle
from typing import Tuple
from sklearn.metrics import confusion_matrix

import numpy as np
import scipy.signal as signal
import wave as wave

from google.colab import drive

/usr/local/lib/python3.6/dist-packages/torchaudio/backend/utils.py:54: UserWarning: "sox" backend is being deprecated. The default backend will be changed to "sox_io" backend in 0.8.0 and "sox" backend will be removed in 0.9.0. Please migrate to "sox_io" backend. Please refer to https://github.com/pytorch/audio/issues/903 for the detail.
  '"sox" backend is being deprecated. '


In [3]:
# drive.mount('/content/drive')
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [4]:
class Net(nn.Module):
  def __init__(self, n_in, n_h, n_out):
    super(Net, self).__init__()
    self.l1 = nn.Linear(n_in, n_h)
    self.l2 = nn.Linear(n_h, n_h)
    self.dropout2 = nn.Dropout2d(0.5)
    self.l3 = nn.Linear(n_h, n_out)
    """
    self.d1 = nn.Linear(n_out, n_h2)
    self.d2 = nn.Linear(n_h2, n_h1)
    self.d3 = nn.Linear(n_h1, n_in)
    

  def encoder(self, x):
    h = F.relu(self.l1(x))
    h = F.relu(self.l2(h))
    h = self.dropout2(h)
    h = F.relu(self.l3(h))
    
    return h

  def decoder(self, x):
    h = F.relu(self.d1(x))
    h = F.relu(self.d2(h))
    h = self.dropout2(h)
    h = F.relu(self.d3(h))
    
    return h
  """
  def forward(self, x):
    """
    h = F.relu(self.l1(x))
    h = F.relu(self.l2(h))
    h = self.dropout2(h)
    h = torch.sigmoid(self.l3(h))
    """
    h = torch.sigmoid(self.l1(x))
    h = torch.sigmoid(self.l2(h))
    h = self.dropout2(h)
    h = self.l3(h)
    """
    z = self.encoder(x)
    h = self.decoder(z)
    """
    return h

In [5]:
base_path = "/content/drive/My Drive/"
output_filename = "Output_WF_-5-10dB_20ep_MSE"
output_base_path = os.path.join(base_path, output_filename)

In [6]:
URL = "train_16000"

def load_timit_item(fileid: str,
                    path: str) -> Tuple[Tensor, Tensor]:

    filter_path = os.path.join(path, fileid, "filter.cpickle")    # for WF
    # snr_path = os.path.join(path, fileid, "snr_stft.cpickle")
    # tfmask_path = os.path.join(path, fileid, "tfmask.cpickle")
    # clean_path = os.path.join(path, fileid, "speech.cpickle")       # for DAE
    # mixed_path = os.path.join(path, fileid, "mixed.cpickle")      # for DAE
    mixed_path = os.path.join(path, fileid,  "input_data.cpickle")  # for WF

    
    # Wiener-Filterのフィルタ係数
    
    with open(filter_path, mode='rb') as f:     # for WF
      sn_ratio = pickle.load(f)
    """
    with open(snr_path, mode='rb') as f:
      sn_ratio = pickle.load(f)

    with open(tfmask_path, mode='rb') as f:
      sn_ratio = pickle.load(f)
    
    with open(clean_path, mode='rb') as f:      # for DAE
      sn_ratio = pickle.load(f)
    """
    with open(mixed_path, mode='rb') as f:
      data = pickle.load(f)

    return (
        data,
        sn_ratio
    )

class TIMIT16000(Dataset):

  def __init__(self,
               root: str,
               url: str = URL,
               download: bool = False) -> None:

      if url in[
            "train_16000",
            "train_16000_0.0dB",
            "train_16000_10.0dB",
            "train_16000_0-5dB",
            "train_16000_-1-4dB",
            "train_16000_-5-10dB",
            "test_16000",
            "test_16000_0.0dB",
            "test_16000_10.0dB",
            "test_16000_0-5dB",
            "test_16000_-1-4dB",
            "test_16000_-5-10dB",
            "demo_data"
      ]:

        base_url = "/content/drive/My Drive/"
        append_url = os.path.join(base_url, url)

      self._path = append_url

      dir_path = os.path.join(self._path, "*" + os.sep)

      self._dir_list = glob.glob(dir_path, recursive=True)

      
      """
      if url in [
                 "train_16000",
                 "train_16000_0.0dB",
                 "train_16000_10.0dB",
                 "train_16000_0-5dB",
                 "train_16000_-1-4dB",
                 "train_16000_-5-10dB"
      ]:
        self._dir_list = self._dir_list[:4616]
      """

      if url in [
                 "test_16000",
                 "test_16000_0.0dB",
                 "test_16000_10.0dB",
                 "test_16000_0-5dB",
                 "test_16000_-1-4dB",
                 "test_16000_-5-10dB",
                 "demo_data"
      ]:
        self._dir_list = self._dir_list[:200]
      
      print(len(self._dir_list))
      print(self._dir_list)

      if url in [
                 "test_16000",
                 "test_16000_0.0dB",
                 "test_16000_10.0dB",
                 "test_16000_0-5dB",
                 "test_16000_-1-4dB",
                 "test_16000_-5-10dB",
                 "demo_data"
      ]:
        list_path = os.path.join(base_path, output_filename, "list.txt")

        with open(list_path, mode='w') as f:
          f.write('\n'.join(self._dir_list))
        
        print("done")


  def __getitem__(self, n: int) -> Tuple[Tensor, Tensor]:
    fileid = self._dir_list[n]

    return load_timit_item(fileid, self._path)

  def __len__(self) -> int:
    return len(self._dir_list)


In [ ]:
def L1_loss(y, label):
  loss = tf.reduce_mean(tf.abs(y - label))
  return loss

def L2_loss(y, label):
  loss = torch.mean((torch.square(torch.abs(y - label))))
  return loss

In [8]:
min_len = 59

epoch = 10
batchsize = 8   #メモリに載るように調整(8とか16ぐらい?)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

torch.manual_seed(0)

wf_train_data = TIMIT16000('data',
                           url = "train_16000_-5-10dB",
                           download=True)

wf_test_data = TIMIT16000('data',
                          url = "test_16000_0.0dB",
                          download=True)

train_loader = torch.utils.data.DataLoader(wf_train_data,
                                           batch_size=batchsize,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(wf_test_data,
                                          batch_size=batchsize,
                                          shuffle=False)

model = Net(1026, 2052, 513).to(device)
optimizer = torch.optim.Adam(model.parameters())

criterion = nn.MSELoss()

18480
['/content/drive/My Drive/train_16000_-5-10dB/4370_-5.00dB/', '/content/drive/My Drive/train_16000_-5-10dB/4370_0.00dB/', '/content/drive/My Drive/train_16000_-5-10dB/4370_5.00dB/', '/content/drive/My Drive/train_16000_-5-10dB/4370_10.00dB/', '/content/drive/My Drive/train_16000_-5-10dB/4371_-5.00dB/', '/content/drive/My Drive/train_16000_-5-10dB/4371_0.00dB/', '/content/drive/My Drive/train_16000_-5-10dB/4371_5.00dB/', '/content/drive/My Drive/train_16000_-5-10dB/4371_10.00dB/', '/content/drive/My Drive/train_16000_-5-10dB/4372_-5.00dB/', '/content/drive/My Drive/train_16000_-5-10dB/4372_0.00dB/', '/content/drive/My Drive/train_16000_-5-10dB/4372_5.00dB/', '/content/drive/My Drive/train_16000_-5-10dB/4372_10.00dB/', '/content/drive/My Drive/train_16000_-5-10dB/4373_-5.00dB/', '/content/drive/My Drive/train_16000_-5-10dB/4373_0.00dB/', '/content/drive/My Drive/train_16000_-5-10dB/4373_5.00dB/', '/content/drive/My Drive/train_16000_-5-10dB/4373_10.00dB/', '/content/drive/My Drive/

In [ ]:
model_path = os.path.join(output_base_path, "learned_model_20ep.pth")
model.load_state_dict(torch.load(model_path))

<All keys matched successfully>

In [ ]:
import requests
import pandas as pd

exportlist = []

export_path = "/content/drive/My Drive/train_0.xlsx"

In [ ]:
model.train()
for ep in range(1, epoch+1):
  tot_loss = 0.0

  for batch_idx, (data, sn_ratio) in enumerate(train_loader):
    # data, sn_ratio = data.to(device), sn_ratio.to(device)

    input = data[0, :, :]
    label = sn_ratio[0, :, :]

    for i in range(1, batchsize):
      input = torch.cat((input, data[i, :, :]), 1)
      label = torch.cat((label, sn_ratio[i, :, :]), 1)

    input = torch.transpose(input, 0, 1)
    label = torch.transpose(label, 0, 1)

    """
    data_zero = input[0, :].to('cpu').detach().numpy().copy()
    sn_ratio_zero = label[0, :].to('cpu').detach().numpy().copy()

    exportlist.append(data_zero)
    exportlist.append(sn_ratio_zero)

    df = pd.DataFrame(exportlist)

    df.T.to_excel(export_path, sheet_name='new_sheet_name')
    print(data[1, 0])

    print("done")
    """
    input, label = input.to(device), label.to(device)

    optimizer.zero_grad()
    output = model(input)

    loss = criterion(output, label)
    # loss = L2_loss(output, label)
    loss.backward()
    optimizer.step()

    tot_loss += loss.item()

  print('Epoch: {:2d}, Average loss: {:.6f}'.format(ep, tot_loss / ((len(train_loader)) * min_len)))

In [ ]:
model_path = os.path.join(output_base_path, "learned_model_10ep.pth")
torch.save(model.state_dict(), model_path)
print("done")

done


In [ ]:
model.eval()
test_loss = 0

with torch.no_grad():
  for i, (data, sn_ratio) in enumerate(test_loader):
    # data, sn_ratio = data.to(device), sn_ratio.to(device)

    input = data[0, :, :]
    label = sn_ratio[0, :, :]

    for j in range(1, batchsize):
      input = torch.cat((input, data[j, :, :]), 1)
      label = torch.cat((label, sn_ratio[j, :, :]), 1)

    input = torch.transpose(input, 0, 1)
    label = torch.transpose(label, 0, 1)

    input, label = input.to(device), label.to(device)

    output = model(input)
    # test_loss += L2_loss(output, sn_ratio)

    output_path = os.path.join(output_base_path, "{:04d}".format(i))
    os.mkdir(output_path)

    output_filter_path = os.path.join(output_path, "filter.cpickle")

    with open(output_filter_path, mode='wb') as f:
      pickle.dump(output, f)

print("done")
# test_loss = test_loss / (len(train_loader)*min_len)
# print("Average loss: {:.4f}".format(test_loss))

done


In [ ]:
wf_demo_data = TIMIT16000('data',
                          url = "demo_data",
                          download=True)

demo_loader = torch.utils.data.DataLoader(wf_demo_data,
                                          batch_size=1,
                                          shuffle=False)

In [ ]:
model.eval()
test_loss = 0

with torch.no_grad():
  for i, (input, label) in enumerate(demo_loader):
    print(np.shape(input))
    print(np.shape(label))

    input = torch.transpose(input, 1, 2)
    label = torch.transpose(label, 1, 2)

    print(np.shape(input))
    print(np.shape(label))

    input, label = input.to(device), label.to(device)

    output = model(input)
    # test_loss += L2_loss(output, sn_ratio)

    output_path = os.path.join(output_base_path, "{:04d}".format(i+100))
    os.mkdir(output_path)

    output_filter_path = os.path.join(output_path, "filter.cpickle")

    with open(output_filter_path, mode='wb') as f:
      pickle.dump(output, f)

In [ ]:
# append

model.eval()
test_loss = 0

with torch.no_grad():
  input_path = "/content/drive/My Drive/input_data.cpickle"
  label_path = "/content/drive/My Drive/train_16000_0.0dB/3620_0.00dB/filter.cpickle"

  with open(input_path, mode='rb') as f:
    input = pickle.load(f)

  with open(label_path, mode='rb') as f:
    label = pickle.load(f)

  input = torch.from_numpy(input.astype(np.float32)).clone()
  print(np.shape(input))
  input = torch.transpose(input, 0, 1)

  # input, label = input.to(device), label.to(device)
  input = input.to(device)

  output = model(input)
  # test_loss += L2_loss(output, sn_ratio)

  output_path = "/content/drive/My Drive/filter.cpickle"

  with open(output_path, mode='wb') as f:
    pickle.dump(output, f)

torch.Size([1026, 227])


In [ ]:
# 実際にフィルタをかけるスペース

fs = 16000
fft_size = 1024
n_overlap = fft_size *0.75

filter_path = "/content/drive/My Drive/Output_WF_0dB_sig_20epoch/0100/filter.cpickle"
wave_path = "/content/drive/My Drive/demo_data/demo.wav"

"""
filter_path = os.path.join(base_path, "filter.cpickle")
wave_path = os.path.join(base_path, "mixed.wav")
"""
mix_src = read_wav(wave_path)

t, f, mix_stft = signal.stft(mix_src, fs=fs, nperseg=fft_size, noverlap=n_overlap, nfft=fft_size, axis=0)

# wave_path = os.path.join(base_path, "mixed.cpickle")

with open(filter_path, mode='rb') as f:
  filter = pickle.load(f)

"""
with open(filter_path, mode='rb') as f:
  mix_wav = pickle.load(f)
"""
print(np.shape(mix_stft))
# print(np.shape(mix_wav))
print(np.shape(filter))

print(mix_stft.dtype)
# print(mix_wav.dtype)

# print(mix_wav[:, 30])
# print(filter[:, 30])

print(np.shape(filter))

# filter = filter[:475, :]

filter = np.einsum('nft->tf', filter)

print(np.shape(filter))

filtered_wav = np.einsum('ft, ft->ft', mix_stft, filter)

t, output_istft = signal.istft(filtered_wav, fs=fs, nperseg=fft_size, noverlap=n_overlap, nfft=fft_size, time_axis=1, freq_axis=0)

output_path = "/content/drive/My Drive/demo_avg_WF_0dB_20ep.wav"

output_wav(output_path, output_istft, wave_path)

NameError: ignored

In [ ]:
# -----Testで上手くいきそうここに復元を実装-----

for i in range(500):
  

In [ ]:
filter_path = "/content/drive/My Drive/Output_WF_10.0dB_20epoch/0000/filter.cpickle"

with open(filter_path, mode='rb') as f:
  filter = pickle.load(f)

print(filter[30, :])
print(np.shape(filter))

tensor([0.2642, 0.2752, 0.2454, 0.2836, 0.3046, 0.2771, 0.2591, 0.2811, 0.2565,
        0.2825, 0.2836, 0.2832, 0.2530, 0.2708, 0.2932, 0.2781, 0.2525, 0.2948,
        0.2936, 0.3029, 0.2797, 0.2918, 0.2613, 0.2692, 0.2834, 0.2813, 0.2384,
        0.2667, 0.2663, 0.2729, 0.2625, 0.2920, 0.2330, 0.2594, 0.2535, 0.2472,
        0.2797, 0.2740, 0.2830, 0.2703, 0.2961, 0.2730, 0.2446, 0.2651, 0.2787,
        0.2600, 0.2782, 0.2490, 0.2615, 0.2471, 0.2570, 0.2769, 0.2834, 0.2505,
        0.2858, 0.2644, 0.2723, 0.2599, 0.2691, 0.2702, 0.2761, 0.2963, 0.3077,
        0.2847, 0.2527, 0.2570, 0.2722, 0.2991, 0.2852, 0.2868, 0.2884, 0.3003,
        0.2867, 0.2807, 0.2537, 0.2641, 0.2640, 0.2884, 0.2664, 0.2760, 0.2767,
        0.2625, 0.2839, 0.2704, 0.2773, 0.2868, 0.2664, 0.2641, 0.2770, 0.2758,
        0.2553, 0.2731, 0.2334, 0.2454, 0.2537, 0.2641, 0.2817, 0.2507, 0.2715,
        0.2697, 0.2783, 0.2546, 0.2871, 0.2708, 0.2587, 0.2608, 0.3109, 0.2927,
        0.2983, 0.3142, 0.3015, 0.2958, 

In [ ]:
 # -----Test Space-----
 
 base_url = "/content/drive/My Drive/"
 url = os.path.join(base_url, "train_16000")
 listlist = glob.glob(os.path.join(url, "*" + os.sep), recursive=True)
 print(len(listlist))

 url = "/content/drive/My Drive/train_16000/0000_1.08dB/"
 mixed_path = os.path.join(url, "mixed.cpickle")
 speech_path = os.path.join(url, "speech.cpickle")

 with open(mixed_path, mode='rb') as f:
   mixed_wave = pickle.load(f)

 with open(speech_path, mode='rb') as f:
   speech_wave = pickle.load(f)

 print(np.shape(mixed_wave))
 print(mixed_wave)

 appended = np.append(mixed_wave, speech_wave, axis=0)
 print(np.shape(appended))

4620
(513, 206)
[[  0.64020286 +0.j           6.92631031 +0.j
   17.55052673 +0.j         ... -12.54181347 +0.j
   -7.10006191 +0.j          -4.51223855 +0.j        ]
 [ -0.34386349 -0.83706301j  -1.16222619 +6.81182872j
  -12.24416282 +3.81978918j ...   7.46667787 +6.70624016j
    1.46318295 -1.41231306j   3.90700279 +3.86028445j]
 [  2.32947443 +1.14293121j  -5.87500936 -6.16711404j
    1.45108882 +2.00033062j ...  -6.73579474 -5.48024793j
    5.37663636+10.36535003j   1.30819115 -7.57482702j]
 ...
 [  0.49054448 -1.77193439j   1.68128821 +8.28153745j
   -4.10512328-10.71742458j ...  -3.71472812+13.55862571j
    6.67297624-14.04013345j  -1.88798108 +6.43104943j]
 [ -1.48808203 -2.66112832j  -9.45669988 -5.2752204j
   -5.63357771 +3.4999775j  ...   1.41568718-12.57620801j
   -6.85239565 +6.64587834j   4.74635917 -0.43153425j]
 [  5.68330519 +0.j          13.53971365 +0.j
   15.63381241 +0.j         ...  -1.45588161 +0.j
    4.30167258 +0.j          -1.69496282 +0.j        ]]
(1026, 20

In [ ]:
mix_path = "/content/drive/My Drive/train_16000_with_snr/4619_5.11dB/snr_stft.cpickle"

with open(mix_path, mode='rb') as f:
  mix_data = pickle.load(f)

print(np.shape(mix_data))
print(mix_data[:, 15])

(513, 48)
[-3.54136914e-01 -2.68722743e-01  5.88244200e-01  4.44563366e-02
 -1.50927806e+00 -4.07888025e-01  1.68676710e+00  2.19709635e+00
  1.85116661e+00 -4.94843572e-01  6.90087020e-01  1.04044831e+00
  7.26287425e-01  8.89382362e-01  1.02638650e+00 -7.46672690e-01
  5.88926613e-01  3.71556729e-01 -1.36971455e+01 -3.58339906e-01
  9.71571326e-01  9.95262206e-01  1.80464268e+00  1.37209595e+02
  4.78995991e+00  5.36131620e-01  7.33951032e-01  9.55840528e-01
  7.86142111e-01  6.68706954e-01  3.34930599e-01  5.05647659e-01
  9.01938438e-01  9.30664301e-01  1.03903151e+00  7.47265041e-01
  2.30936885e+00  8.84274781e-01  9.24942434e-01  9.72401083e-01
  9.07594383e-01  8.70153606e-01  1.11434472e+00 -1.23188332e-01
  5.87603748e-01  8.51429403e-01  9.51294959e-01  1.01224232e+00
  8.16730022e-01  3.46733809e-01  7.69993305e-01  9.91592169e-01
  9.50553358e-01  6.26077175e-01  1.49140465e+00  5.35699606e+00
 -3.25663567e+01  1.80821121e+00  1.51854825e+00  2.08377481e+00
 -8.52155406e-0